In [1]:
#imports for the ML
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import datetime

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [3]:
#Importing csvs in place of sql database

Anx_2020_df = pd.read_csv("Resources/mental_2020_Anxiety_df.csv")

Dep_2020_df = pd.read_csv("Resources/mental_2020_Depression_df.csv")

Either_2020_df = pd.read_csv("Resources/mental_2020_AnxOrDPRS_df.csv")

Anx_2021_df = pd.read_csv("Resources/mental_2021_Anxiety_df.csv")

Dep_2021_df = pd.read_csv("Resources/mental_2021_Depression_df.csv")

Either_2021_df = pd.read_csv("Resources/mental_2021_AnxOrDPRS_df.csv")

In [4]:
le = LabelEncoder()
Anx_2020_df.head()

,Group,Subgroup,Time_Period,Value,Risk
0,By Age,18 - 29 years,2020-04-23,40.2,Extreme
1,By Age,30 - 39 years,2020-04-23,34.4,Moderate
2,By Age,40 - 49 years,2020-04-23,34.1,Moderate
3,By Age,50 - 59 years,2020-04-23,31.0,Moderate
4,By Age,60 - 69 years,2020-04-23,24.9,Mild


In [5]:
def table_func(df):
    
    le = LabelEncoder()
    del df['Value']
    df['Time_Period'] = le.fit_transform(df['Time_Period'])
    

In [6]:
#Changing Time period so it is ML ready, remove "Value"

table_func(Anx_2020_df)

table_func(Dep_2020_df)

table_func(Either_2020_df)

table_func(Anx_2021_df)

table_func(Dep_2021_df)

table_func(Either_2021_df)

Anx_2020_df.head()

,Group,Subgroup,Time_Period,Risk
0,By Age,18 - 29 years,0,Extreme
1,By Age,30 - 39 years,0,Moderate
2,By Age,40 - 49 years,0,Moderate
3,By Age,50 - 59 years,0,Moderate
4,By Age,60 - 69 years,0,Mild


In [7]:
#Create dummy columns
Anx_2020_df = pd.get_dummies(Anx_2020_df, columns=["Group","Subgroup"])

Dep_2020_df = pd.get_dummies(Dep_2020_df, columns=["Group","Subgroup"])

Either_2020_df = pd.get_dummies(Either_2020_df, columns=["Group","Subgroup"])

Anx_2021_df = pd.get_dummies(Anx_2021_df, columns=["Group","Subgroup"])

Dep_2021_df = pd.get_dummies(Dep_2021_df, columns=["Group","Subgroup"])

Either_2021_df = pd.get_dummies(Either_2021_df, columns=["Group","Subgroup"])

Anx_2020_df.head()

,Time_Period,Risk,Group_By Age,Group_By Education,Group_By Race/Hispanic ethnicity,Group_By Sex,Group_By State,Subgroup_18 - 29 years,Subgroup_30 - 39 years,Subgroup_40 - 49 years,...,Subgroup_South Dakota,Subgroup_Tennessee,Subgroup_Texas,Subgroup_Utah,Subgroup_Vermont,Subgroup_Virginia,Subgroup_Washington,Subgroup_West Virginia,Subgroup_Wisconsin,Subgroup_Wyoming
0,0,Extreme,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Moderate,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Moderate,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,Moderate,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,Mild,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Split into x and y

y = Anx_2020_df["Risk"]
X = Anx_2020_df.drop(columns =["Risk"])


In [9]:
X.describe()

,Time_Period,Group_By Age,Group_By Education,Group_By Race/Hispanic ethnicity,Group_By Sex,Group_By State,Subgroup_18 - 29 years,Subgroup_30 - 39 years,Subgroup_40 - 49 years,Subgroup_50 - 59 years,...,Subgroup_South Dakota,Subgroup_Tennessee,Subgroup_Texas,Subgroup_Utah,Subgroup_Vermont,Subgroup_Virginia,Subgroup_Washington,Subgroup_West Virginia,Subgroup_Wisconsin,Subgroup_Wyoming
count,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,...,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000
mean,10.000000,0.101449,0.057971,0.072464,0.028986,0.739130,0.014493,0.014493,0.014493,0.014493,...,0.014493,0.014493,0.014493,0.014493,0.014493,0.014493,0.014493,0.014493,0.014493,0.014493
std,6.057391,0.302027,0.233769,0.259344,0.167824,0.439261,0.119552,0.119552,0.119552,0.119552,...,0.119552,0.119552,0.119552,0.119552,0.119552,0.119552,0.119552,0.119552,0.119552,0.119552
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
y.value_counts()

Moderate    541
Mild        490
High        316
Extreme     102
Name: Risk, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=64)
Counter(y_train)

Counter({'Moderate': 419, 'Extreme': 71, 'Mild': 363, 'High': 233})

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=64)
rf_model = rf_model.fit(X_train, y_train)

In [13]:
# Calculated the balanced accuracy score
pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, pred)

0.6074385902475143

In [14]:
# Display the confusion matrix
confusion_matrix(y_test, pred)

array([[16, 14,  0,  1],
       [ 7, 47,  5, 24],
       [ 0,  3, 92, 32],
       [ 0, 18, 28, 76]], dtype=int64)

In [15]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, pred))

                   pre       rec       spe        f1       geo       iba       sup

    Extreme       0.70      0.52      0.98      0.59      0.71      0.48        31
       High       0.57      0.57      0.88      0.57      0.70      0.48        83
       Mild       0.74      0.72      0.86      0.73      0.79      0.61       127
   Moderate       0.57      0.62      0.76      0.60      0.69      0.47       122

avg / total       0.64      0.64      0.84      0.64      0.73      0.52       363



In [16]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.43668396978931673, 'Time_Period'),
 (0.013878375484774668, 'Subgroup_18 - 29 years'),
 (0.013477387607700647, 'Subgroup_80 years and above'),
 (0.012795042825766115,
  'Subgroup_Non-Hispanic, other races and multiple races'),
 (0.011807955979534511, 'Subgroup_70 - 79 years'),
 (0.011233764034492466, 'Subgroup_Less than a high school diploma'),
 (0.011194065117073364, 'Subgroup_North Dakota'),
 (0.011176364606837103, 'Subgroup_60 - 69 years'),
 (0.010527620794849271, 'Subgroup_Minnesota'),
 (0.010510045932956763, 'Subgroup_New Mexico'),
 (0.010127221719768507, 'Subgroup_Louisiana'),
 (0.00982438423198164, 'Subgroup_Iowa'),
 (0.009781392910632325, 'Subgroup_Non-Hispanic Asian, single race'),
 (0.0096270738420538, 'Subgroup_Nevada'),
 (0.009509385092873307, 'Subgroup_South Dakota'),
 (0.009347879274594387, 'Subgroup_California'),
 (0.00901503284590087, 'Subgroup_Nebraska'),
 (0.008997661680290847, 'Group_By State'),
 (0.008702836638869501, 'Subgroup_Hispanic or Latino'),
 (0.008702685

In [17]:
#Split into x and y for 2021 prediction

y_2021 = Anx_2021_df["Risk"]
X_2021 = Anx_2021_df.drop(columns =["Risk"])

In [18]:
# Calculated the balanced accuracy score
pred = rf_model.predict(X_2021)
balanced_accuracy_score(y_2021, pred)

0.3168896621396964

In [21]:
# Display the confusion matrix
confusion_matrix(y_2021, pred)

array([[ 10,  12,   3,   6],
       [ 18,  28,  43,  51],
       [ 21, 147, 324, 347],
       [ 25,  63, 105, 108]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_2021, pred))

                   pre       rec       spe        f1       geo       iba       sup

    Extreme       0.14      0.32      0.95      0.19      0.55      0.29        31
       High       0.11      0.20      0.81      0.14      0.40      0.15       140
       Mild       0.68      0.39      0.68      0.49      0.51      0.25       839
   Moderate       0.21      0.36      0.60      0.27      0.46      0.21       301

avg / total       0.50      0.36      0.68      0.40      0.49      0.23      1311



In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_2021, y_2021, random_state=64)
Counter(y_train)

Counter({'Mild': 625, 'Moderate': 230, 'High': 104, 'Extreme': 24})

In [24]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=64)
rf_model = rf_model.fit(X_train, y_train)

In [25]:
pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, pred)

0.6066007937552627

In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, pred))

                   pre       rec       spe        f1       geo       iba       sup

    Extreme       0.50      0.43      0.99      0.46      0.65      0.40         7
       High       0.53      0.53      0.94      0.53      0.71      0.48        36
       Mild       0.88      0.88      0.78      0.88      0.83      0.69       214
   Moderate       0.58      0.59      0.88      0.58      0.72      0.51        71

avg / total       0.77      0.77      0.82      0.77      0.79      0.62       328



In [26]:
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.5234651873219903, 'Time_Period'),
 (0.025159633366111114, 'Subgroup_18 - 29 years'),
 (0.02456810194357232,
  'Subgroup_Non-Hispanic, other races and multiple races'),
 (0.02408729353659311, 'Subgroup_30 - 39 years'),
 (0.01331212031193984, 'Subgroup_Louisiana'),
 (0.012305737429259517, 'Subgroup_Less than a high school diploma'),
 (0.01176481302816206, 'Subgroup_Hispanic or Latino'),
 (0.011706992649373894, 'Subgroup_Nevada'),
 (0.011293090750064525, 'Subgroup_Oklahoma'),
 (0.01095909252481687, 'Subgroup_Oregon'),
 (0.009987612117391832, 'Subgroup_West Virginia'),
 (0.009844221598336702, 'Subgroup_Kentucky'),
 (0.00920329509894612, 'Subgroup_Female'),
 (0.008423830624824496, "Subgroup_Some college/Associate's degree"),
 (0.00820003660400136, 'Subgroup_40 - 49 years'),
 (0.008064105149452674, 'Subgroup_Montana'),
 (0.007927100592328859, 'Group_By State'),
 (0.007688832418879379, 'Subgroup_Mississippi'),
 (0.007545111044694427, 'Subgroup_Tennessee'),
 (0.007470402584108058, 'Subgrou